In [22]:
import geopandas as gpd
import pandas as pd
import json
from pathlib import Path

import pandas as pd
from tqdm import tqdm

In [14]:
DATA_PATH = Path("/home/riccardofiorista/Documents/courses/6.C85/fp_code/data")
OUTPUT_PATH = DATA_PATH / "preprocessed_data/parcels/per_station"

In [15]:
parcels = gpd.read_file(DATA_PATH / "preprocessed_data/station_only_parcels.geojson")

KeyboardInterrupt: 

In [19]:
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

def store_station_parcel_data(station_parcels):
    station_only_parcels_to_save = station_parcels.copy().to_crs('EPSG:4326')
    station_only_parcels_to_save['zone_id'] = station_only_parcels_to_save['zone_id'].apply(json.dumps)
    station_only_parcels_to_save['routes'] = station_only_parcels_to_save['routes'].apply(json.dumps)
    station_only_parcels_to_save['route_colors'] = station_only_parcels_to_save['route_colors'].apply(json.dumps)
    
    # Ensure that parcels are for one stop
    stop_names = station_parcels['stop_name'].unique()
    if len(stop_names) > 1:
        raise ValueError(f'Can only process one stop per dataframe not: {stop_names}')
    else:
        stop_name = stop_names[0]
        
    file_name = f"{stop_name}_parcels.geojson"
    file_name = file_name.replace(" ", "_").replace("/", "_").replace(".", "_").lower()
    file_path = OUTPUT_PATH / file_name
    
    station_only_parcels_to_save.to_file(file_path, driver='GeoJSON')
    
    return file_path
    
file_structure = {'StopName': [], 'FileName': []}

for stop_name in tqdm(parcels['stop_name'].unique()):
    parcels_to_store = parcels[parcels['stop_name'] == stop_name]
    file_path = store_station_parcel_data(parcels_to_store)
    file_structure['StopName'].append(stop_name)
    file_structure['FileName'].append(file_path)

100%|██████████| 164/164 [02:46<00:00,  1.01s/it]


In [25]:
file_df = pd.DataFrame(file_structure)

In [69]:
file_df['FileName'] = file_df['FileName'].apply(lambda x: Path(*x.parts[x.parts.index('fp_code') + 1:]))

In [78]:
file_df['FileName'] = file_df['FileName'].astype(str)

In [81]:
pd.DataFrame(file_structure).to_csv(OUTPUT_PATH / 'file_name_reference.csv')